# "SQL Practice"
> "Solving murder mysteries one query at a time"

- toc: false
- branch: master
- badges: true
- comments: true
- categories: [sql, jupyter]
- image: images/sql/sql-server.png

Let's solve a mystery hidden in an sql database. 

The instructions and database was created by Joon Park and Cathy He while fellows at the Knight Lab at Northwesten University. 

Here's a step by step solution, though if you'd like to try it yourself first, 
check out [their interactive site](https://mystery.knightlab.com/).

Here's the description from the knight lab:

> A crime has taken place and the detective needs your help. The detective gave you the crime scene report, but you somehow lost it. You vaguely remember that the crime was a __murder__ that occurred sometime on __Jan.15, 2018__ and that it took place in __SQL City__. Start by retrieving the corresponding crime scene report from the police department’s database. 

### The setup

We'll use pandas to make the queries look nice and sqlite3 to make the connection.

I'm also adding a function pq to quickly grab the results of a query based on the value of the query variable. 

In [1]:
import pandas as pd
import sqlite3
from sqlite3 import Error
from pathlib import Path

In [2]:
def create_connection(path):

    connection = None

    try:

        connection = sqlite3.connect(path)

        print("Connection to SQLite DB successful")

    except Error as e:

        print(f"The error '{e}' occurred")

    return connection

In [3]:
db = Path("../datasets/sql-murder/sql-murder-mystery.db")

conn = create_connection(db)

Connection to SQLite DB successful


Taking a look at all the tables by selecting _name_ filtering by _table_. 

In [4]:
def pq(query, conn=conn):
    """Returns a sql query as a pandas dataframe"""
    return pd.read_sql(query, conn)

In [5]:
query = """SELECT name 
  FROM sqlite_master
 where type = 'table'"""
pq(query)

,name
0,crime_scene_report
1,drivers_license
2,person
3,facebook_event_checkin
4,interview
5,get_fit_now_member
6,get_fit_now_check_in
7,income
8,solution


In order to determine the structure of each table, we can run a query like this:

In [6]:
query = """SELECT sql 
  FROM sqlite_master
 where name = 'crime_scene_report'"""
print(pq(query).iloc[0, 0])

CREATE TABLE crime_scene_report (
        date integer,
        type text,
        description text,
        city text
    )


### Solving the mystery

#### Finding the witnesses

In [7]:
query = """
SELECT *
FROM crime_scene_report
LIMIT 5"""
pq(query)

,date,type,description,city
0,20180115,robbery,A Man Dressed as Spider-Man Is on a Robbery Spree,NYC
1,20180115,murder,Life? Dont talk to me about life.,Albany
2,20180115,murder,"Mama, I killed a man, put a gun against his he...",Reno
3,20180215,murder,REDACTED REDACTED REDACTED,SQL City
4,20180215,murder,Someone killed the guard! He took an arrow to ...,SQL City


Let's start with what we know. According to the prompt, we know that this murder took place on Jan. 15 2018 in SQL City. This should be enough info to explore crime scene reports. 

We care about the description, and want to filter by __city__, __date__, and __type__.

In [8]:
query = """
SELECT description 
FROM crime_scene_report 
WHERE city = 'SQL City' AND type = 'murder' AND date = 20180115"""

print(pq(query).iloc[0, 0])

Security footage shows that there were 2 witnesses. The first witness lives at the last house on "Northwestern Dr". The second witness, named Annabel, lives somewhere on "Franklin Ave".


#### Finding the interviews

Located witnesses presumably interviewed by detectives. 
Let's take a quick look the interview table

In [9]:
query = """
SELECT * 
FROM interview
LIMIT 5
"""

pq(query)

,person_id,transcript
0,28508,‘I deny it!’ said the March Hare.\n
1,63713,\n
2,86208,"way, and the whole party swam to the shore.\n"
3,35267,"lessons in here? Why, there’s hardly room for ..."
4,33856,\n


Not too useful (yet). We need to figure out how the __person_id__ connects 

In [10]:
query = """SELECT sql 
  FROM sqlite_master
 where name = 'interview'"""
print(pq(query).iloc[0, 0])

CREATE TABLE interview (
        person_id integer,
        transcript text,
        FOREIGN KEY (person_id) REFERENCES person(id)
    )


AHA! So let's connect person_id from the person table.

In [11]:
query = """
SELECT sql 
FROM sqlite_master
WHERE name = 'person'"""
print(pq(query).iloc[0, 0])

CREATE TABLE person (
        id integer PRIMARY KEY,
        name text,
        license_id integer,
        address_number integer,
        address_street_name text,
        ssn integer,
        FOREIGN KEY (license_id) REFERENCES drivers_license(id)
    )


The clues we have are:
 *  Annabel, who lives on "Franklin Ave"
 *  Someone else who lives on the last house on "Northwestern Dr"
 
Once we join both tables, a subquery will help capture the last house by finding the largest street number for Northwestern Dr. 
We can use an fstring to inject the subquery, making it a bit more readable. 

In [12]:
subq = """
SELECT MAX(address_number) 
FROM person 
WHERE address_street_name = 'Northwestern Dr'
"""

query = f"""
SELECT transcript
FROM person AS p
INNER JOIN interview AS i ON p.id = i.person_id
WHERE (address_street_name = 'Franklin Ave' AND name LIKE 'Annabel%') OR (address_street_name = 'Northwestern Dr' AND address_number IN ({subq}));
"""
[print(v) for v in pq(query)["transcript"]]

I heard a gunshot and then saw a man run out. He had a "Get Fit Now Gym" bag. The membership number on the bag started with "48Z". Only gold members have those bags. The man got into a car with a plate that included "H42W".
I saw the murder happen, and I recognized the killer from my gym when I was working out last week on January the 9th.


[None, None]

Let's extract the salient points: 

* Suspect was at the gym on __Jan 9th__

* Suspect (same?) __male__ and has __Get Fit Now Bag__, presumably a member with __"48Z"__ as the starting membership id. __Gold member__ bag. Car connected to suspect has plate number partial of __"H42W"__

Taking a look at gym data:

In [13]:
query = """SELECT sql 
  FROM sqlite_master
 where name = 'get_fit_now_check_in'"""
print(pq(query).iloc[0, 0])

CREATE TABLE get_fit_now_check_in (
        membership_id text,
        check_in_date integer,
        check_in_time integer,
        check_out_time integer,
        FOREIGN KEY (membership_id) REFERENCES get_fit_now_member(id)
    )


In [14]:
query = """SELECT sql 
  FROM sqlite_master
 where name = 'get_fit_now_member'"""
print(pq(query).iloc[0, 0])

CREATE TABLE get_fit_now_member (
        id text PRIMARY KEY,
        person_id integer,
        name text,
        membership_start_date integer,
        membership_status text,
        FOREIGN KEY (person_id) REFERENCES person(id)
    )


In [15]:
query = """
SELECT *
FROM get_fit_now_check_in
WHERE check_in_date = 20180109 AND membership_id LIKE '48Z%'
"""
pq(query)

,membership_id,check_in_date,check_in_time,check_out_time
0,48Z7A,20180109,1600,1730
1,48Z55,20180109,1530,1700


This is useful. Let's see if we can combine this with the info from the other witness to isolate a single suspect.

In [16]:
subq = """
SELECT *
FROM get_fit_now_check_in
WHERE check_in_date = 20180109 AND membership_id LIKE '48Z%'
"""

query = f"""
SELECT *
FROM ({subq}) AS c
INNER JOIN get_fit_now_member AS m ON m.id = c.membership_id
"""
pq(query)

,membership_id,check_in_date,check_in_time,check_out_time,id,person_id,name,membership_start_date,membership_status
0,48Z7A,20180109,1600,1730,48Z7A,28819,Joe Germuska,20160305,gold
1,48Z55,20180109,1530,1700,48Z55,67318,Jeremy Bowers,20160101,gold


We can take advantage of the 'free' filtration by performing an inner join on the membership id. 

Unfortunately we're stuck between two people. Our last resort is the driver's license. 

In [17]:
query = """SELECT sql 
  FROM sqlite_master
 where name = 'drivers_license'"""
print(pq(query).iloc[0, 0])

CREATE TABLE drivers_license (
        id integer PRIMARY KEY,
        age integer,
        height integer,
        eye_color text,
        hair_color text,
        gender text,
        plate_number text,
        car_make text,
        car_model text
    )


In [18]:
query = """SELECT sql 
  FROM sqlite_master
 where name = 'person'"""
print(pq(query).iloc[0, 0])

CREATE TABLE person (
        id integer PRIMARY KEY,
        name text,
        license_id integer,
        address_number integer,
        address_street_name text,
        ssn integer,
        FOREIGN KEY (license_id) REFERENCES drivers_license(id)
    )


Lots of interesting info in these tables, so lets select everything that has a pretty good match. 

In [19]:
query = """
SELECT *
FROM drivers_license
WHERE plate_number LIKE '%H42W%' AND gender = 'male'
"""
pq(query)

,id,age,height,eye_color,hair_color,gender,plate_number,car_make,car_model
0,423327,30,70,brown,brown,male,0H42W2,Chevrolet,Spark LS
1,664760,21,71,black,black,male,4H42WR,Nissan,Altima


Interesting... lets see if it matches either of our gym members. 

In [20]:
subq = """
SELECT *
FROM drivers_license
WHERE plate_number LIKE '%H42W%' AND gender = 'male'
"""

query = f"""
SELECT *
FROM ({subq}) d
INNER JOIN person p ON d.id = p.license_id"""

pq(query)

,id,age,height,eye_color,hair_color,gender,plate_number,car_make,car_model,id,name,license_id,address_number,address_street_name,ssn
0,664760,21,71,black,black,male,4H42WR,Nissan,Altima,51739,Tushar Chandra,664760,312,Phi St,137882671
1,423327,30,70,brown,brown,male,0H42W2,Chevrolet,Spark LS,67318,Jeremy Bowers,423327,530,"Washington Pl, Apt 3A",871539279


Ah! We found the suspect in the list, but let's create a query that combines it all.

In [21]:
g_subq = """
SELECT *
FROM get_fit_now_check_in
WHERE check_in_date = 20180109 AND membership_id LIKE '48Z%'
"""

g_query = f"""
SELECT name
FROM ({g_subq}) AS c
INNER JOIN get_fit_now_member AS m ON m.id = c.membership_id
"""


subq = """
SELECT *
FROM drivers_license
WHERE plate_number LIKE '%H42W%' AND gender = 'male'
"""

query = f"""
SELECT *
FROM ({subq}) d
INNER JOIN person p ON d.id = p.license_id
WHERE name IN ({g_query})
"""
pq(query)

,id,age,height,eye_color,hair_color,gender,plate_number,car_make,car_model,id,name,license_id,address_number,address_street_name,ssn
0,423327,30,70,brown,brown,male,0H42W2,Chevrolet,Spark LS,67318,Jeremy Bowers,423327,530,"Washington Pl, Apt 3A",871539279


Perhaps we found the murderer? Let's see if there's anything else we can find out about the suspect.

In [22]:
query = """
SELECT *
FROM interview
WHERE person_id = 67318
"""
print(pq(query).iloc[0, 1])

I was hired by a woman with a lot of money. I don't know her name but I know she's around 5'5" (65") or 5'7" (67"). She has red hair and she drives a Tesla Model S. I know that she attended the SQL Symphony Concert 3 times in December 2017.



Interesting.. turns out we're not done yet!

### Finding the real murderer

* Between 65" AND 67"
* Red hair
* High income 
* Tesla Model S
* Checkin SQL Symphony Concert 3 times in December 2017

In [23]:
query = """
SELECT * 
FROM drivers_license 
WHERE height BETWEEN 65 AND 67 
    AND car_make = 'Tesla' 
    AND gender = 'female' 
    AND hair_color = 'red'

"""

pq(query)

,id,age,height,eye_color,hair_color,gender,plate_number,car_make,car_model
0,202298,68,66,green,red,female,500123,Tesla,Model S
1,291182,65,66,blue,red,female,08CM64,Tesla,Model S
2,918773,48,65,black,red,female,917UU3,Tesla,Model S


In [24]:
query = """SELECT sql 
  FROM sqlite_master
 where name = 'facebook_event_checkin'"""
print(pq(query).iloc[0, 0])

CREATE TABLE facebook_event_checkin (
        person_id integer,
        event_id integer,
        event_name text,
        date integer,
        FOREIGN KEY (person_id) REFERENCES person(id)
    )


In [25]:
query = """SELECT sql 
  FROM sqlite_master
 where name = 'income'"""
print(pq(query).iloc[0, 0])

CREATE TABLE income (
        ssn integer PRIMARY KEY,
        annual_income integer
    )


We can create a single query and combine all 

In [26]:
query = """
SELECT name, annual_income
FROM facebook_event_checkin f
INNER JOIN person p ON p.id = f.person_id 
INNER JOIN drivers_license d ON p.license_id = d.id
INNER JOIN income i ON p.ssn = i.ssn
WHERE date BETWEEN 20171200 AND 20171231
    AND event_name = 'SQL Symphony Concert'
    AND height BETWEEN 65 AND 67
    AND hair_color = 'red'
    AND gender = 'female'
    AND car_make = 'Tesla'
    AND car_model = 'Model S'
GROUP BY person_id
HAVING COUNT(person_id) > 2
"""

pq(query)

,name,annual_income
0,Miranda Priestly,310000


Looks like a match!